This notebook gathers the data collected from our buffers and alternative buffers contructed in the notebooks `Construct-Buffers-v17`, `Construct-Buffers-v16`, `Construct-AltBuffers-v17.ipynb`, and `Construct-AltBuffers-v16` and combines it with Dicken's own dataset.

Note that each of the above-mentioned notebooks require the Ethnologe to run which is protected under copy rights. Therefore, these notebooks can't be run here on Deepnote. However, you can see the outputs of each cell. If you want to understand how the data was constructed please review the notebooks.

You need to change the variable `buffer_size_radius_km` equal to the radius of the buffer you want to create for notebooks `Construct-Buffers-v16` and `Construct-AltBuffers-v16`. To replicate the Dickens (2022) buffers you need to set up `buffer_size_radius_km=50` so that the buffers have a 100km diameter.

In [21]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys, os, time
pd.set_option('display.width', 140)

from IPython.display import display, HTML, Image

# pathdata = '/work/data/'

# Do not use these lines in DEEPNOTE
pathdata = '../data/'

Make sure to decompress the dataset `Dickens_AltBuf_v16_crops_absdif_50.zip` to get the dataset `Dickens_AltBuf_v16_crops_absdif_50.dta`.

## Preparing the Final Dataset

In [32]:
# Load all datasets
dfor = pd.read_stata(pathdata + 'EJ_Dickens_Border_100km.dta')

dfor_v16 = pd.read_stata(pathdata + 'Dickens_OrBuf_v16_stats_50.dta') # this naming convention is with the radius not the diameter
dfor_v16_25 = pd.read_stata(pathdata + 'Dickens_OrBuf_v16_stats_25.dta')
dfor_v16_100 = pd.read_stata(pathdata + 'Dickens_OrBuf_v16_stats_100.dta')

dfor_v16_alt = pd.read_stata(pathdata + 'Dickens_AltBuf_v16_absdif_50.dta')
dfor_v16_alt_25 = pd.read_stata(pathdata + 'Dickens_AltBuf_v16_absdif_25.dta')
dfor_v16_alt_100 = pd.read_stata(pathdata + 'Dickens_AltBuf_v16_absdif_100.dta')

# Load crop specific datasets
dfor_crops = pd.read_stata(pathdata + 'Dickens_OrBuf_v16_cropstats_50.dta')
dfor_crops_alt = pd.read_stata(pathdata + 'Dickens_AltBuf_v16_crops_absdif_50.dta')

# Redefine some variables to be closer to what Dickens did with buffers defined by Dickens
for df in [dfor_v16, dfor_v16_25, dfor_v16_100]:
    df['csi_change_sd_oj'] = (df.post1500AverageCaloriesstd - df.pre1500AverageCaloriesstd)/1000
    df['csi_sd_oj'] = (df.pre1500AverageCaloriesstd)/1000
    df['csi_change_oj'] = (df.post1500AverageCaloriesmean - df.pre1500AverageCaloriesmean)/1000
    df['csi_oj'] = (df.pre1500AverageCaloriesmean)/1000

# Redefine some variables to be closer to what Dickens did with the alternative buffers
for df in [dfor_v16_alt,dfor_v16_alt_25, dfor_v16_alt_100]:
    df['csi_change_alt'] = (df.post1500AverageCaloriesmean - df.pre1500AverageCaloriesmean)/1000
    df['csi_alt'] = (df.pre1500AverageCaloriesmean)/1000

C:\Users\48672590\AppData\Local\Temp\ipykernel_14104\935551799.py:2: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dfor = pd.read_stata(pathdata + 'EJ_Dickens_Border_100km.dta')


In [33]:
# add the same for version v17 of the Ethnologe
dfor_v17 = pd.read_stata(pathdata + 'Dickens_OrBuf_v17_stats.dta')
dfor_v17_alt = pd.read_stata(pathdata + 'Dickens_AltBuf_v17_absdif.dta')

dfor_v17['csi_change_sd_oj'] = (dfor_v17.post1500AverageCaloriesstd - dfor_v17.pre1500AverageCaloriesstd)/1000
dfor_v17['csi_sd_oj'] = (dfor_v17.pre1500AverageCaloriesstd)/1000
dfor_v17['csi_change_oj'] = (dfor_v17.post1500AverageCaloriesmean - dfor_v17.pre1500AverageCaloriesmean)/1000
dfor_v17['csi_oj'] = (dfor_v17.pre1500AverageCaloriesmean)/1000

# Redefine some variables to be closer to what Dickens did with the alternative buffers
dfor_v17_alt['csi_change_alt'] = (dfor_v17_alt.post1500AverageCaloriesmean - dfor_v17_alt.pre1500AverageCaloriesmean)/1000
dfor_v17_alt['csi_alt'] = (dfor_v17_alt.pre1500AverageCaloriesmean)/1000

In [34]:
# Redefine some variables to be easier to work with
crop_vars = ['crop_' + col if col != 'identifier' else col for col in dfor_crops.columns]
dfor_crops.columns = crop_vars
crop_vars.remove('identifier')

crop_alt_vars = ['crop_alt_' + col  if col not in ['identifier','ID_1','ID_2'] else col for col in dfor_crops_alt.columns]
dfor_crops_alt.columns = crop_alt_vars
crop_alt_vars.remove('identifier')
crop_alt_vars.remove('ID_1')
crop_alt_vars.remove('ID_2')

# Now we need to normalize all the crop data
for col in dfor_crops.columns:
    if col == 'identifier':
        continue
    dfor_crops[col] = dfor_crops[col]/1000

for col in dfor_crops_alt.columns:
    if col in ['identifier','ID_1','ID_2']:
        continue
    dfor_crops_alt[col] = dfor_crops_alt[col]/1000

In [35]:
# To include some same language pairs 
dfor_v16_all = dfor_v16.merge(dfor, how = 'left', on = 'identifier')
dfor_v16_all = dfor_v16_all.merge(dfor_v16_alt, how = 'left', on = 'identifier')

In [36]:
# We want to make sure that we have the same as buffers zones Dicken, 
# so we will first restrict our sample to those that have information about lingDist
dfor = dfor[dfor.lingDist.isna() == False]

# Merge both the alternative measures and the original reconstruction
dfor_v16 = dfor.merge(dfor_v16, how = 'left', on = 'identifier')
dfor_v16 = dfor_v16.merge(dfor_v16_alt, how = 'left', on = 'identifier')
dfor_v16 = dfor_v16.merge(dfor_crops, how = 'left', on = 'identifier')
dfor_v16 = dfor_v16.merge(dfor_crops_alt, how = 'left', on = 'identifier')

dfor_v16_25 = dfor.merge(dfor_v16_25, how = 'left', on = 'identifier')
dfor_v16_25 = dfor_v16_25.merge(dfor_v16_alt_25, how = 'left', on = 'identifier')

dfor_v16_100 = dfor.merge(dfor_v16_100, how = 'left', on = 'identifier')
dfor_v16_100 = dfor_v16_100.merge(dfor_v16_alt_100, how = 'left', on = 'identifier')

# Merge with ethnologe v17
dfor_v17 = dfor.merge(dfor_v17, how = 'left', on = 'identifier')
dfor_v17 = dfor_v17.merge(dfor_v17_alt, how = 'left', on = 'identifier')

In [37]:
# We need to test if we consider those borders that share the same language like Mexico and Guatemala which both uses Spanish.
same_language_pairs = pd.read_stata(pathdata + 'same_language_identifiers.dta')

dfor_v16 = dfor_v16.merge(same_language_pairs, on='identifier', how='left')
dfor_v16_25 = dfor_v16_25.merge(same_language_pairs, on='identifier', how='left')
dfor_v16_100 = dfor_v16_100.merge(same_language_pairs, on='identifier', how='left')
dfor_v16_all = dfor_v16_all.merge(same_language_pairs, on='identifier', how='left')
dfor_v17 = dfor_v17.merge(same_language_pairs, on='identifier', how='left')

In [38]:
# Modify the dfor_v16_all and add the missing data of LingDist only to those that we identify as same language
dfor_v16_all.loc[dfor_v16_all['same_lang'] == 1, 'lingDist'] = 0

In [39]:
# To get the same number of observations in the regressions we also need to identify family1 and family2 singletons
# For 25km buffers
category_counts = dfor_v16_25.loc[dfor_v16_25.lingDist.isna()==False].groupby('family1').identifier.count()
singletop_fam1 = category_counts[category_counts == 1].index.tolist()

category_counts = dfor_v16_25.loc[dfor_v16_25.lingDist.isna()==False].groupby('family2').identifier.count()
singletop_fam2 = category_counts[category_counts == 1].index.tolist()

dfor_v16_25['not_singletons'] = (~dfor_v16.family1.isin(singletop_fam1)) & (~dfor_v16_25.family2.isin(singletop_fam2))
print(sum(dfor_v16_25.csi.isna() == False))
print(sum(dfor_v16_25.not_singletons == 1))

# For 50km buffers
dfor_v16['not_singletons'] = (~dfor_v16.family1.isin(singletop_fam1)) & (~dfor_v16.family2.isin(singletop_fam2))
print(sum(dfor_v16.csi_alt.isna() == False))
print(sum(dfor_v16.not_singletons == 1))

# For 100km buffers
dfor_v16_100['not_singletons'] = (~dfor_v16_100.family1.isin(singletop_fam1)) & (~dfor_v16_100.family2.isin(singletop_fam2))
print(sum(dfor_v16_100.csi_alt.isna() == False))
print(sum(dfor_v16_100.not_singletons == 1))

# For all borders
category_counts = dfor_v16_all.loc[dfor_v16_all.lingDist.isna()==False].groupby('family1').identifier.count()
singletop_fam1 = category_counts[category_counts == 1].index.tolist()

category_counts = dfor_v16_all.loc[dfor_v16_all.lingDist.isna()==False].groupby('family2').identifier.count()
singletop_fam2 = category_counts[category_counts == 1].index.tolist()
dfor_v16_all['not_singletons'] = (~dfor_v16_all.family1.isin(singletop_fam1)) & (~dfor_v16_all.family2.isin(singletop_fam2))
print(dfor_v16_all.shape)
print(sum(dfor_v16_all.not_singletons == 1))

8426
8402
8426
8402
8426
8402
(67025, 412)
66964


C:\Users\48672590\AppData\Local\Temp\ipykernel_14104\2076897082.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_counts = dfor_v16_25.loc[dfor_v16_25.lingDist.isna()==False].groupby('family1').identifier.count()
C:\Users\48672590\AppData\Local\Temp\ipykernel_14104\2076897082.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_counts = dfor_v16_25.loc[dfor_v16_25.lingDist.isna()==False].groupby('family2').identifier.count()
C:\Users\48672590\AppData\Local\Temp\ipykernel_14104\2076897082.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a futur

In [43]:
# Do the same for v17
category_counts = dfor_v17.loc[dfor_v17.lingDist.isna()==False].groupby('family1').identifier.count()
singletop_fam1 = category_counts[category_counts == 1].index.tolist()

category_counts = dfor_v17.loc[dfor_v17.lingDist.isna()==False].groupby('family2').identifier.count()
singletop_fam2 = category_counts[category_counts == 1].index.tolist()

dfor_v17['not_singletons'] = (~dfor_v17.family1.isin(singletop_fam1)) & (~dfor_v17.family2.isin(singletop_fam2))
print(sum(dfor_v17.csi.isna() == False))
print(sum(dfor_v17.not_singletons == 1))

8431
8407


C:\Users\48672590\AppData\Local\Temp\ipykernel_14104\2182301182.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_counts = dfor_v17.loc[dfor_v17.lingDist.isna()==False].groupby('family1').identifier.count()
C:\Users\48672590\AppData\Local\Temp\ipykernel_14104\2182301182.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_counts = dfor_v17.loc[dfor_v17.lingDist.isna()==False].groupby('family2').identifier.count()


In [44]:
dfor_v16['alt_subset'] = dfor_v16.identifier != 'BZX-MLI-FFM-MLI'
dfor_v16_25['alt_subset'] = dfor_v16_25.identifier != 'BZX-MLI-FFM-MLI'
dfor_v16_100['alt_subset'] = dfor_v16_100.identifier != 'BZX-MLI-FFM-MLI'
dfor_v17['alt_subset'] = dfor_v17.identifier != 'BZX-MLI-FFM-MLI'

In [45]:
# The border 'XMW-MDG-PLT-MDG' is in buffers 25 and 100 but not in 50 so lets remove it from the 25 and 100
dfor_v16_25 = dfor_v16_25[dfor_v16_25.identifier != 'XMW-MDG-PLT-MDG']
dfor_v16_100 = dfor_v16_100[dfor_v16_100.identifier != 'XMW-MDG-PLT-MDG']

In [46]:
# Get the variables used in the analysis and order them with the csi variables 1st
var_to_keep = dfor.columns.tolist()
var_to_keep.remove("csi")
var_to_keep.remove("csi_sd")
var_to_keep.remove("csi_change")
var_to_keep.remove("csi_change_sd")

var_to_keep = ['csi','csi_sd','csi_change','csi_change_sd','csi_oj','csi_change_oj','csi_sd_oj','csi_change_sd_oj','csi_alt','csi_change_alt',
               'same_lang','not_singletons','alt_subset'] \
              + var_to_keep
var_to_keep_enhanced = var_to_keep + crop_vars + crop_alt_vars

dfor_v16[var_to_keep_enhanced].to_stata(pathdata + 'Dickens_rep_v16_50.dta', version=117)
dfor_v16_25[var_to_keep].to_stata(pathdata + 'Dickens_rep_v16_25.dta', version=117)
dfor_v16_100[var_to_keep].to_stata(pathdata + 'Dickens_rep_v16_100.dta', version=117)
dfor_v17[var_to_keep].to_stata(pathdata + 'Dickens_rep_v17.dta', version=117)

In [47]:
var_to_keep = dfor.columns.tolist()
var_to_keep.remove("csi")
var_to_keep.remove("csi_sd")
var_to_keep.remove("csi_change")
var_to_keep.remove("csi_change_sd")

var_to_keep = ['csi','csi_sd','csi_change','csi_change_sd','csi_oj','csi_change_oj','csi_sd_oj','csi_change_sd_oj','same_lang','not_singletons'] + var_to_keep
dfor_v16_all[var_to_keep].to_stata(pathdata + 'Dickens_rep_v16_50_allpairs.dta', version=117)

# Testing